In [19]:
import yfinance as yf
import pandas as pd
import psycopg2


In [2]:
# Define the stocks to pull data from
stocks = ['AMZN', 'TSLA', 'NVDA', 'AAPL', 'MSFT', 'META']

# Adjust period & interval to pull data from (1 day, 5 min)
period = '1d'
interval= '5m'
date_format = "%b-%d-%y %H:%M"

In [3]:
stock_data = {}

for stock in stocks:
    df = yf.download(stock, period=period, interval=interval) # Set interval, currently 5 minute intervals
    
    # Check if data is empty
    if not df.empty:
        df.index = df.index.strftime(date_format) # Standardise datetime format
        df.reset_index(inplace=True)
    
    # Store in dictionary
    stock_data[stock] = df

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [13]:
# Input DB params
params = {
    'dbname': 
    'user':
    'password': 
    'host': 
    'port': 
}

conn = psycopg2.connect(**params)
cur = conn.cursor()

insert_query = """
INSERT INTO price (company_id, date, open_price, close_price, high_price, low_price, volume)
VALUES (%s, %s, %s, %s, %s, %s, %s);
"""

In [11]:
# Extract company name and sector from yahoo finance for companies not in database
def get_stock_info(ticker_symbol):
    stock = yf.Ticker(ticker_symbol)
    
    try:
        name = stock.info['longName']
        sector = stock.info['sector']
        return {'Name': name, 'Sector': sector}
    except KeyError as e:
        return f"Could not find '{e.args[0]}' information for ticker symbol {ticker_symbol}"
    

# Get the company_id if it exists, add the company_id if not
def get_or_create_company_id(ticker_symbol, cur):
    # Check if the company exists
    cur.execute("SELECT company_id FROM company WHERE ticker_symbol = %s", (ticker_symbol,))
    result = cur.fetchone()
    
    if result:
        return result  # Return existing company_id
    else:
        # Get company info
        company_info = get_stock_info(ticker_symbol)
        cur.execute("INSERT INTO company (company_name, ticker_symbol, sector) VALUES (%s, %s, %s) RETURNING company_id",
                    (company_info['Name'], ticker_symbol, company_info['Sector']))
        company_id = cur.fetchone()[0]
        return company_id

In [15]:
for stock, df in stock_data.items():
    for _, row in df.iterrows():
        # Get company id
        company_id = get_or_create_company_id(stock, cur)
    
        # Prepare data for insertion
        data_tuple = (
            company_id,  # Using the stock symbol directly
            pd.to_datetime(row['Datetime'], format=date_format),
            row['Open'],
            row['Close'],
            row['High'],
            row['Low'],
            row['Volume']
        )
        # Execute INSERT command
        cur.execute(insert_query, data_tuple)

# Commit the transaction
conn.commit()

In [17]:
conn = psycopg2.connect(**params)
cur = conn.cursor()
cur.execute("SELECT * FROM company")
result = cur.fetchall()
print(result)

[(2, 'Amazon', 'AMZN', 'Tech'), (3, 'TSLA', 'TSLA', 'Tech'), (4, 'NVDA', 'NVDA', 'Tech'), (5, 'AAPL', 'AAPL', 'Tech'), (6, 'MSFT', 'MSFT', 'Tech'), (7, 'META', 'META', 'Tech')]
